In [ ]:
using PyCall
using PyPlot
using Statistics

yf = pyimport("yfinance")

include("PortfolioOptimization.jl")

# Data donwload and basic analysis 

In [ ]:
start = "2013-01-01"
finish = "2017-01-01" 

start_test = "2017-01-01"
finish_test  ="2018-01-01"

assets = ["MSFT", "AAPL", "TSLA", "GOOG", "META", "AMZN"] #"^GSPC"

df = yf.download(assets, start, finish, progress=false)

df_test = yf.download(assets, start_test, finish_test, progress=false)

fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
df["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

df_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
df["Adj Close"].pct_change().plot(ax=ax["C"])

ax["C"].legend(ncol=2)

df_test["Adj Close"].pct_change().plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
for asset in assets
    df["Adj Close"][asset].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
end

ax["E"].legend(ncol=2)

for asset in assets
    df_test["Adj Close"][asset].pct_change().hist(ax=ax["F"], bins=50, alpha=0.8, label=asset)
end

ax["F"].legend(ncol=2)

# Resample data to 5-day frequency

In [ ]:
df = df.resample("5d").mean()

df_test = df_test.resample("5d").mean()

fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
df["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

df_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
df["Adj Close"].pct_change().plot(ax=ax["C"])

ax["C"].legend(ncol=2)

df_test["Adj Close"].pct_change().plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
for asset in assets
    df["Adj Close"][asset].pct_change().hist(ax=ax["E"], bins=30, alpha=0.8, label=asset)
end

ax["E"].legend(ncol=2)

for asset in assets
    df_test["Adj Close"][asset].pct_change().hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end

ax["F"].legend(ncol=2)

# Mean-Variance portfolio optimization 

In [ ]:
μ = df["Adj Close"].pct_change().mean().values

Σ = df["Adj Close"].pct_change().cov().values

portfolio = Portfolio(µ, Σ)

target_return = 0.006

w_opt = MPT_fixed_return(portfolio, target_return)

println("Expected Returns: ", sum(w_opt.*μ))
println("Risk: ", sqrt(transpose(w_opt)*portfolio.Σ*w_opt))
println("Weights:", w_opt)

In [ ]:
μ = df["Adj Close"].pct_change().mean().values

Σ = df["Adj Close"].pct_change().cov().values

portfolio = Portfolio(µ, Σ)

target_risk = 0.018

w_opt = MPT_fixed_risk(portfolio, target_risk)

println("Expected Returns: ", sum(w_opt.*μ))
println("Risk: ", sqrt(transpose(w_opt)*portfolio.Σ*w_opt))
println("Weights:", w_opt)

# Efficient frontier 

In [ ]:
MeanVar, weights = Efficient_frontier(portfolio, 100);

risks = sqrt.(diag(portfolio.Σ))

for i in 1:length(portfolio.μ)

    plt.scatter(risks[i], portfolio.μ[i], s=100, label=assets[i])
    
end

plt.plot(sqrt.(MeanVar[:, 2]), MeanVar[:, 1], marker=".")

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

# Portfolio testing 

We select the portfolio with target_return of 0.005 

In [ ]:
μ = df["Adj Close"].pct_change().mean().values

Σ = df["Adj Close"].pct_change().cov().values

portfolio = Portfolio(µ, Σ)

df_test_returns = df_test["Adj Close"].pct_change()

target_return = 0.004

w_opt = MPT_fixed_return(portfolio, target_return)

println("Expected Returns: ", sum(w_opt.*μ))
println("Risk: ", sqrt(transpose(w_opt)*portfolio.Σ*w_opt))
println("Weights:", w_opt)

## Obtain yields after 5-day investment period for all posible investments next year

In [ ]:
returns = df_test_returns.values[2:length(df_test_returns), :] * w_opt
    
risk = round(sqrt(var(returns)), digits=2)

dates = [df_test["Adj Close"].index[i] for i in 2 : length(df_test)]

mean_return = round(mean(returns), digits=4)

plt.figure(figsize=(8,6))

plt.plot(dates, returns)

plt.axhline(mean_return, color="k", lw=3, label="μ=$mean_return,  σ=$risk")

plt.legend()

In [ ]:
μ = df["Adj Close"].pct_change().mean().values

Σ = df["Adj Close"].pct_change().cov().values

portfolio = Portfolio(µ, Σ)

df_test_returns = df_test["Adj Close"].pct_change()

target_returns = 0.003 : 0.00025 : 0.007

final_risks = []
final_returns = []

expected_risks = []
expected_returns = []

@time for target_return in target_returns

    w_opt = MPT_fixed_return(portfolio, target_return)

    expected_return = sum(w_opt.*μ)
    expected_risk = sqrt(transpose(w_opt)*portfolio.Σ*w_opt)

    returns = df_test_returns.values[2:length(df_test_returns), :] * w_opt

    actual_risk = round(sqrt(var(returns)), digits=6)
    actual_mean_return = round(mean(returns), digits=6)
            
    append!(expected_risks, expected_risk)
    append!(expected_returns, expected_return)
    
    append!(final_risks, actual_risk)
    append!(final_returns, actual_mean_return)
    
end

plt.plot(expected_risks, expected_returns, marker="o", label="Expected Frontier")
plt.plot(final_risks, final_returns, marker="o", label="Real frontier")

plt.legend()

# TO DO 

- Compare results with market investment (S&P500)